In [1]:
import tensorflow as tf 
import tensorflow.keras as keras 
import numpy as np

In [2]:
with open('인간문제.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
len(text)

239261

In [4]:
text[-20:]

'(1934.8.1~12.22)\n \n '

In [5]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(lower=False, char_level=True) 
tokenizer.fit_on_texts(text)

In [6]:
len(tokenizer.word_index)

1247

max_id : 고유 글자 개수 (1,247개) 

dataset_size: 전체 글자 개수 (239,261개)

In [7]:
max_id = len(tokenizer.word_index) # number of distinct characters
dataset_size = tokenizer.document_count # total number of characters

In [8]:
print(max_id)
print(dataset_size)

1247
239261


전체 텍스트를 Encoding해서 각 글자를 ID로 나타내기 

(0부터 시작하기 위해서 1 빼기) 

In [9]:
[encoded] = np.array(tokenizer.texts_to_sequences([text])) - 1

텍스트의 처음 90%를 훈련 세트로 사용 

이 텍스트 세트에서 한 번에 한 글자씩 반환하는 tf.data.Dataset 객체 만들기 

In [10]:
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

window() method를 사용해서 여러 텍스트 window로 변환 

100(모델이 보는 데이터) + 1(모델이 맞추어야 하는 데이터) 

RNN은 이 부분 문자열 길이만큼만 Back-propagation 

Truncated BackPropagation Through Time(TBPTT) 

In [11]:
n_steps = 100
window_length = n_steps + 1 # target <= input shifted 1 character ahead
dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True)

모델의 실제 입력으로 사용되는 tensor 형태로 변환 

window마다 batch(window_length)를 호출 

In [12]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [13]:
np.random.seed(42)
tf.random.set_seed(42)

In [14]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

One-hot encoding 

In [15]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

dataset = dataset.prefetch(1)

(batch_size, n_steps, max_id) (batch_size, n_steps) 

In [16]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 1247) (32, 100)


In [23]:
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(128, return_sequences=True, input_shape=[None, max_id], dropout=0.2),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.LSTM(64, return_sequences=True, dropout=0.2),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.LSTM(32, return_sequences=True, dropout=0.2),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dense(max_id, activation="softmax")
])

model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", lr = 0.01, metrics=['accuracy'])

In [24]:
history = model.fit(dataset, steps_per_epoch=train_size // batch_size, epochs=30)

Train for 6729 steps
Epoch 1/30
6729/6729 [==============================] - 158s 24ms/step - loss: 4.5294 - accuracy: 0.2528
Epoch 2/30
6729/6729 [==============================] - 153s 23ms/step - loss: 4.5070 - accuracy: 0.2529
Epoch 3/30
6729/6729 [==============================] - 154s 23ms/step - loss: 4.5070 - accuracy: 0.2529
Epoch 4/30
6729/6729 [==============================] - 153s 23ms/step - loss: 4.5069 - accuracy: 0.2529
Epoch 5/30
6729/6729 [==============================] - 153s 23ms/step - loss: 4.5069 - accuracy: 0.2529
Epoch 6/30
6729/6729 [==============================] - 153s 23ms/step - loss: 4.5069 - accuracy: 0.2529
Epoch 7/30
6729/6729 [==============================] - 153s 23ms/step - loss: 4.5069 - accuracy: 0.2529
Epoch 8/30
6729/6729 [==============================] - 154s 23ms/step - loss: 4.5067 - accuracy: 0.2529
Epoch 9/30
6729/6729 [==============================] - 153s 23ms/step - loss: 4.5064 - accuracy: 0.2529
Epoch 10/30
6729/6729 [===========

In [25]:
# Check its architecture
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, None, 128)         704512    
_________________________________________________________________
activation_3 (Activation)    (None, None, 128)         0         
_________________________________________________________________
lstm_4 (LSTM)                (None, None, 64)          49408     
_________________________________________________________________
activation_4 (Activation)    (None, None, 64)          0         
_________________________________________________________________
lstm_5 (LSTM)                (None, None, 32)          12416     
_________________________________________________________________
activation_5 (Activation)    (None, None, 32)          0         
_________________________________________________________________
dense_1 (Dense)              (None, None, 1247)       

모델 사용하기

모델에 새로운 텍스트 입력을 위한 전처리 

In [26]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [27]:
X_new = preprocess(["이놈의 계집애, 깜작 말고 서"])
Y_pred = model.predict_classes(X_new)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # 1st sentence, last char

' '

In [28]:
tf.random.set_seed(42)
# https://www.tensorflow.org/versions/r2.1/api_docs/python/tf/random/categorical 
tf.random.categorical([[np.log(0.5), np.log(0.4), np.log(0.1)]], num_samples=5).numpy()

array([[0, 0, 1, 1, 1]], dtype=int64)

다음 글자를 온도에 따라 선택 

온도가 매우 높으면 모든 글자가 동일한 확률을 가짐 

In [31]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [32]:
tf.random.set_seed(42)

next_char("이놈의 계집애, 깜작 말고 서", temperature=1)

' '

In [33]:
next_char("이놈의 계집애, 깜작 말고 서", temperature=0.1)

' '

다음 글자를 반복적으로 얻어서 텍스트에 추가 

In [36]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [50]:
tf.random.set_seed(42)

print(complete_text("이", temperature=0.1))

이                                                  


In [38]:
print(complete_text("이", temperature=0.25))

이                                                  


In [39]:
print(complete_text("이", temperature=0.5))

이,    에             르바             "               


In [40]:
print(complete_text("이", temperature=0.75))

이 니어
 골 에이 이
    다   어 .   을 .덤   .  고  수   선냐가그"  


In [41]:
print(complete_text("이", temperature=1))

이 져을니렸 을잠떻자첫. 는써마 소그야그봐넣 는을떡첫그로쁜도맛집,그얼"이거  장 ,우먹모 을


In [42]:
print(complete_text("이", temperature=2))

이져부인려방수바파에먹순넣가팥강었심용군주각배듣연솟들픽벌다원엄숨타하않듯린신면등…기벽감핑런
뜰머밴


In [43]:
print(complete_text("깜", temperature=0.2))

깜                                                  


In [44]:
print(complete_text("깜", temperature=0.25))

깜                                                  


In [45]:
print(complete_text("깜", temperature=0.5))

깜는                       다에                        


In [46]:
print(complete_text("깜", temperature=0.75))

깜  가      다이  어 오 으 문.  아 .무서자내히   기  이     고같는 끼 "


In [47]:
print(complete_text("깜", temperature=1))

깜.며다에어.히숙   러,다격할테는익쳐니일   에  표생  짐짐 계이 에으러밤 부표젖 은  


In [48]:
print(complete_text("깜", temperature=2))

깜되껏"를저서듯머줄起도생
먹쌈협이뒤랐문보려갑합게 궐간장않므보윗 대살솟발탔았깊땀얼쳐 의니지말의


In [49]:
print(complete_text("나무", temperature=0.25))

나무                                                  
